In [1]:
import requests
import json
import os
import sqlalchemy
import pandas as pd
import mdapi_functions as md
from IPython.display import display, Markdown
import markdown
from bokeh.plotting import figure, show
from bokeh.models import (Span, TabPanel, Tabs, ColumnDataSource, DataCube,
                          GroupingInfo, StringFormatter, SumAggregator,
                          TableColumn, HoverTool)
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.io import output_notebook
from math import pi
from datetime import datetime
import datacite_api_functions as dcf
import numpy as np
import data_doc_helper as dh

from math import pi


In [2]:
output_notebook(hide_banner=True)

x = {
    'United States': 157,
    'United Kingdom': 93,
    'Japan': 89,
    'China': 63,
    'Germany': 44,
    'India': 42,
    'Italy': 40,
    'Australia': 35,
    'Brazil': 32,
    'France': 31,
    'Taiwan': 31,
    'Spain': 29,
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [3]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["ethnicity"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [4]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["sex"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [5]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["year_of_birth"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [6]:
x = "10.83126/ukllc-dataset-00067-01"
md.make_hlink("https://api.test.datacite.org/dois/{}/activities".format(x), x + "/activities")

' <a href="https://api.test.datacite.org/dois/10.83126/ukllc-dataset-00067-01/activities" rel="noopener noreferrer" target="_blank">10.83126/ukllc-dataset-00067-01/activities</a>'

In [7]:
md.make_hlink("https://doi.org/" + x, x)

' <a href="https://doi.org/10.83126/ukllc-dataset-00067-01" rel="noopener noreferrer" target="_blank">10.83126/ukllc-dataset-00067-01</a>'

In [8]:
df = md.get_md_api_dsvs()

In [9]:
df2 = md.prep_dsvs_for_gb_pages()

In [10]:
df2[df2["source_table"] == "GENSCOT_SMOKING"]

,source,source_table,version_num,table,version_date,table_name,short_desc,long_desc,collection_start,collection_end,topic_tags,source_name,research_grouping,participants_invited,participants_included,num_rows,num_columns,Owner,geographic_coverage_Nations,geographic_coverage_Regions
169,GENSCOT,GENSCOT_SMOKING,2,SMOKING,20220302.0,Smoking,,Socio-demographic and clinical data were obtai...,01/2006,05/2011,,Generation Scotland: Scottish Family Health St...,"Centre for Medical Informatics, Usher Institut...",21552,20944,21552.0,12.0,University of Edinburgh,Scotland,Nationwide


In [13]:
df = md.get_md_api_dsvs()
df[df["source"] == "MCS"]

,table_full,index,table,description_table,version_date,num_rows,source,values_table,version_num,num_columns,num_participants
194,COVID_CM_w1_v0001_20211101,194,COVID_CM_w1,COVID_CM_w1_v0001_description_20211101,20211101.0,2640.0,MCS,COVID_CM_w1_v0001_values_20211101,v0001,304.0,2542.0
195,COVID_CM_w2_v0001_20211101,195,COVID_CM_w2,COVID_CM_w2_v0001_description_20211101,20211101.0,3266.0,MCS,COVID_CM_w2_v0001_values_20211101,v0001,781.0,3107.0
196,COVID_CM_w3_v0001_20211101,196,COVID_CM_w3,COVID_CM_w3_v0001_description_20211101,20211101.0,4464.0,MCS,COVID_CM_w3_v0001_values_20211101,v0001,828.0,4338.0
197,COVID_parent_w1_v0001_20211101,197,COVID_parent_w1,COVID_parent_w1_v0001_description_20211101,20211101.0,2829.0,MCS,COVID_parent_w1_v0001_values_20211101,v0001,304.0,2675.0
198,COVID_parent_w2_v0001_20211101,198,COVID_parent_w2,COVID_parent_w2_v0001_description_20211101,20211101.0,5702.0,MCS,COVID_parent_w2_v0001_values_20211101,v0001,781.0,5310.0
...,...,...,...,...,...,...,...,...,...,...,...
272,mcs7_hhgrid_v0001_20211101,272,mcs7_hhgrid,mcs7_hhgrid_v0001_description_20211101,20211101.0,22458.0,MCS,mcs7_hhgrid_v0001_values_20211101,v0001,67.0,15346.0
273,mcs7_parent_cm_interview_v0001_20211101,273,mcs7_parent_cm_interview,mcs7_parent_cm_interview_v0001_description_202...,20211101.0,14940.0,MCS,mcs7_parent_cm_interview_v0001_values_20211101,v0001,58.0,12001.0
274,mcs7_parent_derived_v0001_20211101,274,mcs7_parent_derived,mcs7_parent_derived_v0001_description_20211101,20211101.0,12679.0,MCS,mcs7_parent_derived_v0001_values_20211101,v0001,17.0,10468.0
275,mcs7_parent_interview_v0001_20211101,275,mcs7_parent_interview,mcs7_parent_interview_v0001_description_20211101,20211101.0,12679.0,MCS,mcs7_parent_interview_v0001_values_20211101,v0001,281.0,10468.0
